# CO-OPS API For Data Retrieval



*   https://api.tidesandcurrents.noaa.gov/api/prod/
*   https://tidesandcurrents.noaa.gov/stations.html?type=Historic+Water+Levels#Texas




In [ ]:
!pip install noaa_coops

In [ ]:
from noaa_coops import Station, get_stations_from_bbox
import folium
from pprint import pprint
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd
from pprint import pprint

In [ ]:
def retrieve_and_save_station_data(station_id):
    base_path = '/content/drive/MyDrive/Cedar Lakes Project/Data/Water Gauge Data/'

    # Initialize station object
    station = Station(id=station_id)

    # Print station details (optional)
    print(f"Station Name: {station.name}")
    print(f"Latitude, Longitude: {station.lat_lon}")
    pprint(station.data_inventory)

    # Get the start and end dates for 'Verified 6-Minute Water Level'
    start_date = station.data_inventory['Verified Hourly Height Water Level']['start_date']
    end_date = station.data_inventory['Verified Hourly Height Water Level']['end_date']

    # Convert dates to "YYYYMMDD" format
    start_date_formatted = datetime.strptime(start_date, '%Y-%m-%d %H:%M').strftime('%Y%m%d')
    end_date_formatted = datetime.strptime(end_date, '%Y-%m-%d %H:%M').strftime('%Y%m%d')

    # Retrieve water levels
    df_water_levels = station.get_data(
        begin_date=start_date_formatted,
        end_date=end_date_formatted,
        product="hourly_height",
        datum="MSL",
        units="metric",
        time_zone="gmt"
    )

    # Rename and dropna for water levels
    df_water_levels.rename(columns={'v': 'water_level'}, inplace=True)

    # Retrieve wind data
    df_wind_data = station.get_data(
        begin_date=start_date_formatted,
        end_date=end_date_formatted,
        product="wind",
        units="metric",
        time_zone="lst_ldt",
        interval='h'
    )

    # Rename and dropna for wind data
    df_wind_data.rename(columns={'d': 'wind_direction', 's': 'wind_speed', 'dr': 'direction'}, inplace=True)

    # Merge dataframes
    merged_df = pd.merge(df_water_levels, df_wind_data, left_index=True, right_index=True, how='outer')

    merged_df.rename(columns={'t': 'time'}, inplace=True)

    # Save merged data to Excel
    file_path = f'{base_path}water_guage_{station_id}.xlsx'
    merged_df.to_excel(file_path)

    return merged_df


### Map of Station Data Near Targeted Location: Cedar Lake, Texas

In [ ]:

"""
Cedar Lake, Texas:
    Latitude: 28.9011° N
    Longitude: -95.6369° W
"""
# Define the bounding box coordinates
lat_coords = [27, 30]
lon_coords = [-94, -99]

# Retrieve stations from the bounding box
# stations = get_stations_from_bbox(lat_coords, lon_coords)

# stations = [8771972, 8772440, 8772447, 8772471, 8773767, 8772479, 8772391]
stations = [8771972, 8772440, 8772447, 8772471, 8773767]

# Initialize a folium map. Start with the first station's location or a default location
if stations:
    first_station = Station(id=stations[0])
    start_location = [first_station.lat_lon['lat'], first_station.lat_lon['lon']]
else:
    start_location = [28.5, -96.5]  # Default location if no stations are found

map = folium.Map(location=start_location, zoom_start=7)

# Iterate over each station and add it to the map
# for station_id in stations:
for station_id in stations:

    station = Station(id=station_id)
    latitude = station.lat_lon['lat']
    longitude = station.lat_lon['lon']
    station_name = station.name
    folium.Marker([latitude, longitude], popup=station_name).add_to(map)

# Display the map
map


### Availability Analysis of Station Data Near Target Location (Cedar Lake, Texas)

In [ ]:
data = []
# for station_id in stations:
for station_id in stations:
    station = Station(id=station_id)
    start_date_str = station.data_inventory['Verified Hourly Height Water Level']['start_date']
    end_date_str = station.data_inventory['Verified Hourly Height Water Level']['end_date']
    start_year = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M').year
    end_year = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M').year
    station_name = station.name
    data.append({'station_name': station_name, 'station_id': station_id, 'start_year': start_year, 'end_year': end_year })

df = pd.DataFrame(data)
df['station_url'] = df['station_id'].apply(lambda id: f"https://www.tidesandcurrents.noaa.gov/stationhome.html?id={id}")
df


In [ ]:
stations = [8772440, 8772447, 8772471, 8773767]

for station_id in stations:
    retrieve_and_save_station_data(station_id)


In [ ]:

stations = [8771972]

for station_id in stations:
    retrieve_and_save_station_data(station_id)

https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=hourly_height&application=NOS.COOPS.TAC.WL&begin_date=19540101&end_date=20083112&datum=MLLW&station=8772440&time_zone=GMT&units=english&format=json

In [ ]:
https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=hourly_height&application=NOS.COOPS.TAC.WL&begin_date=19540101&end_date=19550101&datum=NAVD&station=8772440&time_zone=GMT&units=metric&format=json